In [3]:
import os
import re
import pandas as pd
import numpy as np
import time
import random
from tqdm import tqdm
import PyPDF2
import requests
import json


from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

 

In [4]:
service = Service(GeckoDriverManager().install())

C:\Users\pawan\AppData\Local\Temp\ipykernel_19904\952567057.py:1: DeprecationWarning: Firefox will soon stop logging to geckodriver.log by default; Specify desired logs with log_output
  service = Service(GeckoDriverManager().install())


In [5]:
headers_list = [
    # Firefox 77 Mac
    {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
    },
    # Chrome 92.0 Win10
    {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
    },
    # Chrome 91.0 Win10
    {
    "Connection": "keep-alive",
    "DNT": "1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Dest": "document",
    "Referer": "https://www.google.com/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
    },
    # Firefox 90.0 Win10
    {
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-User": "?1",
    "Sec-Fetch-Dest": "document",
    "Referer": "https://www.google.com/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9"
    }
]

In [6]:
firefox_options = Options()
#firefox_options.add_argument('--headless')
firefox_options.add_argument('--no-sandbox')
firefox_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Firefox(service=service, options=firefox_options)

In [7]:
def scroll_down(url):
    global driver
    
    driver.get(url) 
    button = driver.find_element(By.XPATH, value ='//button[normalize-space()="See all reviews"]')
    button.click()
    button2 = driver.find_element(By.ID, 'sortBy_1')
    button2.click()
    actions = ActionChains(driver)
    actions.send_keys(Keys.DOWN).send_keys(Keys.DOWN).send_keys(Keys.ENTER)
    actions.perform()
    while True:
        actions = ActionChains(driver)
        actions.send_keys(Keys.DOWN).send_keys(Keys.DOWN).send_keys(Keys.DOWN).send_keys(Keys.DOWN).send_keys(Keys.DOWN).send_keys(Keys.DOWN).send_keys(Keys.DOWN).send_keys(Keys.DOWN).send_keys(Keys.DOWN).send_keys(Keys.DOWN).perform()
        

In [ ]:
# Run for each url and then gather the reviews and ratings
url = 'https://play.google.com/store/apps/details?id=com.gap.mobile.gap&hl=en_US&gl=US'
# url = 'https://play.google.com/store/search?q=banana+republic&c=apps&hl=en_US&gl=US'
# url = 'https://play.google.com/store/search?q=old+navy&c=apps&hl=en_US&gl=US'
# url = 'https://play.google.com/store/search?q=zara&c=apps&hl=en_US&gl=US'
# url = 'https://play.google.com/store/apps/details?id=com.hm.goe'
# url = 'https://play.google.com/store/apps/details?id=com.levistrauss.customer&hl=en_US&gl=US'


 """WARNING!!! THE scroll_down FUNCTION RUNS AN INFINITE LOOP. 
    PLEASE MONITOR THE DRIVER AND INTERRUPT THE KERNEL 
    MANUALLY UPON REACHING THE END OF REVIEWS PANE"""
scroll_down(url)

In [9]:
# once we scroll to the end of the reviews pane we gather the reviews and ratings
reviews = driver.find_elements(By.XPATH, value= "//div[contains(@class, 'RHo1pe')]")
ratings = driver.find_elements(By.XPATH, value= "//div[contains(@class, 'iXRFPc')]")
master_data = []
for review, rating in zip(reviews, ratings[3:]):
    review_eles = review.text.split("\n")
    review_date = review_eles[2]
    review_text = review_eles[3]
    rating = rating.get_attribute('aria-label')
    rating_no = re.findall('\d{1}', rating)[0]
    master_data.append([review_date, review_text, rating_no])

In [10]:
len(reviews)

160

In [11]:
len(ratings)

163